# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features.values
y = boston.target

## Train test split

Perform a train-test-split with a test set of 0.20.

In [3]:
from sklearn.model_selection import train_test_split

In [5]:
train_test_split?

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Fit the model and apply the model to the make test set predictions

In [4]:
from sklearn.linear_model import LinearRegression

Calculate the residuals and the mean squared error

In [8]:
model = LinearRegression()

In [9]:
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [10]:
model.coef_

array([-2.01540533e-01,  8.76989583e-03, -2.39105786e-02,  2.70319668e+00,
       -2.35658326e+01,  2.35462006e+00,  1.18930633e-02, -1.74910336e+01,
        3.48962766e-01, -1.28872762e-02, -9.04104403e-01,  2.46516674e+00,
       -5.36312041e+00])

In [11]:
y_pred = model.predict(X_test)

In [12]:
residuals = y_test - y_pred

In [13]:
squared_error = residuals ** 2

In [16]:
mse = squared_error.mean()

In [17]:
mse

23.439652977777545

In [14]:
len(boston_features)

506

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [43]:
506%20

6

In [6]:
def kfolds(data, k):
    # Force data as pandas dataframe
    # add 1 to fold size to account for leftovers 
    folds = []
    df_size = len(data.index)
    first_few = df_size % k
    fold_size = df_size//k
    start = 0
    
    for i in range(1,k+1):
        if i <= first_few:
            end = start+fold_size+1
            fold = data.loc[start:end].copy()
            first_few -= 1
        else:
            end = start+fold_size
            fold = data.loc[start:end].copy()
        folds.append(fold)
        start = end
    return folds

### Apply it to the Boston Housing Data

In [12]:
# Make sure to concatenate the data again
y2 = pd.DataFrame(boston.target,columns=['medv'])
X2 = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
# boston_df = pd.merge(boston_features,boston_target,left_index=True,right_index=True)
boston_df = pd.concat([X2.reset_index(drop=True), y2], axis=1)
boston_df.tail()

,CHAS,RM,DIS,B,LSTAT,medv
501,0.0,6.593,0.331081,0.987619,-0.169811,22.4
502,0.0,6.120,0.297277,1.000000,-0.274682,20.6
503,0.0,6.976,0.274575,1.000000,-1.067939,23.9
504,0.0,6.794,0.315551,0.991301,-0.836660,22.0
505,0.0,6.030,0.335545,1.000000,-0.510809,11.9


In [13]:
k_folds = kfolds(boston_df,5)
k_folds[0].loc[:,:'LSTAT'].head()

,CHAS,RM,DIS,B,LSTAT
0,0.0,6.575,0.542096,1.000000,-1.275260
1,0.0,6.421,0.623954,1.000000,-0.263711
2,0.0,7.185,0.623954,0.989737,-1.627858
3,0.0,6.998,0.707895,0.994276,-2.153192
4,0.0,7.147,0.707895,1.000000,-1.162114


### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [14]:
# X2 = boston_features.values
# y2 = boston.target
from sklearn.metrics import mean_squared_error
train = pd.concat([fold for i, fold in enumerate(k_folds) if i!=0])
train[y2.columns].head()

,medv
102,18.6
103,19.3
104,20.1
105,19.5
106,19.5


In [15]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(k_folds) if i!=n])
    test = k_folds[n]
    # Fit a linear regression model
    # X_train,X_test,y_train,y_test = train_test_split(train,test,test_size=.2)
    model = LinearRegression()
    model.fit(train[X2.columns],train[y2.columns])
    #Evaluate Train and Test Errors
    y_pred = model.predict(test[X2.columns])
    y_train_pred = model.predict(train[X2.columns])
    
    res_test = y_pred-test[y2.columns]
    res_train = y_train_pred-train[y2.columns]
    
    test_errs.append(np.mean(res_test.astype(float)**2))
    train_errs.append(np.mean(res_train.astype(float)**2))

train_mean = np.array(train_errs).mean()   
test_mean = np.array(test_errs).mean()

print(train_errs)
print(test_errs)
#print(train_mean/test_mean)
# plt.scatter(list(range(k)), train_errs, label='Training Error')
# plt.scatter(list(range(k)), test_errs, label='Testing Error')
# plt.legend();

[medv    24.598434
dtype: float64, medv    23.203756
dtype: float64, medv    20.241954
dtype: float64, medv    15.451048
dtype: float64, medv    22.733846
dtype: float64]
[medv    13.467698
dtype: float64, medv    18.573022
dtype: float64, medv    36.343976
dtype: float64, medv    59.452355
dtype: float64, medv    24.358852
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [16]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
linreg = LinearRegression()

cv_5_results = cross_val_score(linreg, X, y, cv=5,scoring="neg_mean_squared_error")
cv_5_results

array([-13.0161921 , -14.62832183, -24.81432997, -55.24107773,
       -19.022338  ])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [19]:
cv_5_results.mean()

-25.344451926139904

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!